In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Lang API key: ")

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

page_content: a string representing the content;
metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information.

Note that an individual Document object often represents a chunk of a larger document.


In [2]:
from langchain_core.documents import Document

# 5 documents based on 3 different sources

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

Vector search is a common way to store and search over unstructured data (such as unstructured text)

LangChain VectorStore objects contain methods for adding text and Document objects to the store, and querying them using various similarity metrics. They are often initialized with embedding models, which determine how text data is translated to numeric vectors.

We'll use Chroma, which includes an in memory implementation.

In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# chroma as vector store
# openai embeddings are used to embed the documents
# (it's an embedding model that specify how text should be converted into a numeric vector)
vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

In [ ]:
# Chroma here returns a distance metric that should vary inversely with similarity.
vectorstore.similarity_search_with_score("cats")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.2983378469944),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.39783185720443726),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.4192977845668793),
 (Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  0.427507609128952)]

In [ ]:
vectorstore.similarity_search_with_relevance_scores("cats")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.6981951843579781),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.6944375820789894),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.6506818408464647),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.6479584168628121)]

In [ ]:
embedding = OpenAIEmbeddings().embed_query("cats")
print(embedding)
vectorstore.similarity_search_by_vector(embedding)

[0.021665092557668686, -0.011226201429963112, 0.009110305458307266, -0.015605615451931953, -0.02009047195315361, 0.03095816634595394, -0.02156667970120907, -0.024814333766698837, -0.0021967364009469748, -0.0028469704557210207, 0.01003117673099041, 0.006435559131205082, -0.009651580825448036, -0.011816684156656265, -0.01708182319998741, -0.004108775872737169, 0.038184549659490585, 0.009208719246089458, 0.01903603971004486, -0.014284058474004269, -0.0058450764045119286, -0.0013602192047983408, 0.007359945680946112, 0.006182495038956404, -0.00785201508551836, 0.010003059171140194, -0.00198233500123024, -0.011437088251113892, 0.01211895514279604, -0.007381034549325705, 0.02619212679564953, -0.024758098646998405, 0.0021194112487137318, -0.030761338770389557, -0.02645925059914589, -0.0035077487118542194, 0.0043934728018939495, -0.017911309376358986, 0.010488098487257957, -0.01654757559299469, 0.008611206896603107, 0.023703664541244507, -0.005876709241420031, 0.01327180303633213, -0.005300285

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

In [ ]:
from langchain_core.runnables import RunnableLambda

# we want to use similarity_search to retrieve documents
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

# distinct queries,dDefault implementation runs invoke in parallel using a thread pool executor.
retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them.

In [14]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

Retrievers can easily be incorporated into more complex applications, such as retrieval-augmented generation (RAG) applications that combine a given question with retrieved context into a prompt for a LLM. Below we show a minimal example.

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model

In [17]:
response = rag_chain.invoke(input("Ask something about animals: "))
print(response.content)

Cats are independent pets that often enjoy their own space.
